### Schritt 1: Daten Vorbereiten


1. Das Datensatz ausgleichen. Die Daten sind unbalanciert, was bedeutet, dass es viel mehr Beispiele für eine Klasse gibt als für die andere. Dies kann zu einem Modell führen, das nicht gut generalisiert. Wir können die Daten ausgleichen, indem wir die Anzahl der Beispiele in jeder Klasse anpassen. In diesem Fall verwenden wir die `RandomOverSampler`-Klasse von `imblearn` um die Daten auszugleichen.
2. Die Daten normalisieren. Wir verwenden die `StandardScaler`-Klasse von `sklearn` um die Daten zu normalisieren. Dies ist wichtig, da einige Algorithmen empfindlich auf die Skala der Daten reagieren.
3. Die Daten in Trainings- und Testdaten aufteilen. Wir verwenden die `train_test_split`-Funktion von `sklearn` um die Daten in Trainings- und Testdaten aufzuteilen. Dies ist wichtig, um sicherzustellen, dass das Modell auf neuen, unsichtbaren Daten getestet wird.
4. Die Daten in Tensoren umwandeln. Wir verwenden die `torch.tensor`-Funktion von `torch` um die Daten in Tensoren umzuwandeln. Dies ist wichtig, da PyTorch mit Tensoren arbeitet.

In [12]:
import numpy as np
from sklearn import preprocessing 
import tensorflow as tf

In [2]:
#die Daten einlesen
csv = np.loadtxt("Audiobooks_data.csv", delimiter=",")
input = csv[:, 1:-1] #Spalte 0 ID und letzte Spalte target entfernen
target = csv[:, -1] #letzte Spalte target

In [3]:
#Daten mischen
shuffled_indices = np.arange(input.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_input = input[shuffled_indices]
shuffled_target = target[shuffled_indices]

In [4]:
#Target Verteilung checken
target_1 = np.sum(shuffled_target)
print(f"converted macht {target_1/shuffled_target.shape[0]*100}% der Daten aus")

converted macht 15.88327179778472% der Daten aus


In [5]:
#Daten ausgleichen durch entfernen von 0er Werten
zero_target_counter = 0
indices_to_remove = []
for i in range(shuffled_target.shape[0]):
    if shuffled_target[i] == 0:
        zero_target_counter += 1
        if zero_target_counter > target_1:
            indices_to_remove.append(i)

balance_input = np.delete(shuffled_input, indices_to_remove, axis=0)
balance_target = np.delete(shuffled_target, indices_to_remove, axis=0)


In [6]:
#Target Verteilung checken
target_1 = np.sum(balance_target)
print(f"converted macht {target_1/balance_target.shape[0]*100}% der Daten aus")

converted macht 50.0% der Daten aus


In [7]:
#Daten standardisieren
scaled_input = preprocessing.scale(balance_input) 

In [8]:
#nochmal Daten mischen, sonst ist die target Verteilung nicht mehr gleichmäßig
shuffled_indices = np.arange(scaled_input.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_scaled_input = scaled_input[shuffled_indices]
shuffled_balance_target = balance_target[shuffled_indices]

In [9]:
#Daten aufteilen in Trainings- und Testdaten
sample_size = shuffled_scaled_input.shape[0]
train_size = int(0.8 * sample_size)
test_size = int(0.1 * sample_size)
validation_size = sample_size - train_size - test_size

train_input = shuffled_scaled_input[:train_size]
train_target = shuffled_balance_target[:train_size]
validation_input = shuffled_scaled_input[train_size:train_size + validation_size]
validation_target = shuffled_balance_target[train_size:train_size + validation_size]
test_input = shuffled_scaled_input[train_size + validation_size:]
test_target = shuffled_balance_target[train_size + validation_size:]

#checken ob die Verteilung der Daten gleich ist
print(np.sum(train_target)/train_size)
print(np.sum(validation_target)/test_size)
print(np.sum(test_target)/validation_size)

0.49958088851634536
0.49440715883668906
0.5089285714285714


### Schritt 2: Model erstellen

In [14]:
input_size = train_input.shape[1]
output_size = 2 #0 und 1
hidden_size = 50 #50 ist eine gute Zahl zu starten, kann aber auch höher sein


In [15]:
#wählen wir aktivierungsfunktionen und loss function
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
    #wir verwenden hier softmax, weil wir eine Wahrscheinlichkeitsverteilung haben wollen
    tf.keras.layers.Dense(output_size, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\test\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
batch_size = 100 #es ist eine gute Zahl zu starten, kann aber auch höher sein
epochs = 100 #es ist eine gute Zahl zu starten, kann aber auch höher sein

In [17]:
#Trainieren des Modells
#batch_size ist die Anzahl der Daten, die in einem Schritt verarbeitet werden
model.fit(train_input, train_target, batch_size=batch_size, epochs=epochs, validation_data=(validation_input, validation_target), verbose=1)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.6070 - loss: 0.6391 - val_accuracy: 0.7500 - val_loss: 0.5072
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7460 - loss: 0.4863 - val_accuracy: 0.7746 - val_loss: 0.4405
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7793 - loss: 0.4432 - val_accuracy: 0.7790 - val_loss: 0.4097
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7783 - loss: 0.4191 - val_accuracy: 0.7835 - val_loss: 0.3956
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7991 - loss: 0.3981 - val_accuracy: 0.8058 - val_loss: 0.3803
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7885 - loss: 0.3911 - val_accuracy: 0.7969 - val_loss: 0.3741
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7990 - loss: 0.3869 - val_accuracy: 0.7991 - val_loss: 0.3662
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7925 - loss: 0.3808 - val_accuracy: 0.7857

In [ ]:
#setzen frühzeitige stop Funktion ein, um zu verhindern, dass das Modell überfitten